In [ ]:
# 优化后的LSTM股价预测模型
# 所有必要的导入
import pandas as pd
import numpy as np
import sqlite3
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# 设备配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")


In [ ]:
# 数据加载和预处理函数
def load_and_preprocess_data(db_path="../getTradeData/binance.db", table_name="BTCUSDT"):
    """加载数据并进行基本预处理"""
    try:
        connection = sqlite3.connect(db_path)
        sql_query = f"SELECT * FROM {table_name}"
        df = pd.read_sql_query(sql_query, connection)
        connection.close()
        
        # 数据类型转换
        numeric_columns = ['open', 'high', 'low', 'close', 'volume', 
                          'quote_asset_volume', 'taker_buy_base_asset_volume', 
                          'taker_buy_quote_asset_volume']
        df[numeric_columns] = df[numeric_columns].astype(float)
        
        print(f"数据加载成功，形状: {df.shape}")
        return df
    except Exception as e:
        print(f"数据加载错误: {e}")
        return None

# 加载数据
df = load_and_preprocess_data()
if df is not None:
    print("前5行数据:")
    print(df.head())


In [ ]:
# 优化的特征工程函数
def calculate_technical_indicators(df):
    """计算技术指标，避免重复和冗余"""
    # 基本技术指标
    # RSI
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    df['rsi'] = 100 - (100 / (1 + rs))
    
    # EMA
    df['ema_12'] = df['close'].ewm(span=12).mean()
    df['ema_26'] = df['close'].ewm(span=26).mean()
    
    # MACD
    df['macd'] = df['ema_12'] - df['ema_26']
    df['macd_signal'] = df['macd'].ewm(span=9).mean()
    df['macd_histogram'] = df['macd'] - df['macd_signal']
    
    # 移动平均线 (选择有代表性的周期)
    df['sma_10'] = df['close'].rolling(window=10).mean()
    df['sma_20'] = df['close'].rolling(window=20).mean()
    
    # 布林带
    df['bb_middle'] = df['sma_20']
    df['bb_std'] = df['close'].rolling(window=20).std()
    df['bb_upper'] = df['bb_middle'] + 2 * df['bb_std']
    df['bb_lower'] = df['bb_middle'] - 2 * df['bb_std']
    
    # 价格相对位置指标 (更有意义的特征)
    df['price_to_sma10'] = df['close'] / df['sma_10']  # 相对比率而非差值
    df['price_to_sma20'] = df['close'] / df['sma_20']
    df['bb_position'] = (df['close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    # 波动率
    df['volatility'] = df['close'].rolling(window=10).std()
    df['price_change'] = df['close'].pct_change()
    
    # 成交量相关指标
    df['volume_ma'] = df['volume'].rolling(window=10).mean()
    df['volume_ratio'] = df['volume'] / df['volume_ma']
    
    # 高低点指标
    df['hl_ratio'] = (df['high'] - df['low']) / df['close']
    df['oc_ratio'] = (df['close'] - df['open']) / df['open']
    
    print("技术指标计算完成")
    return df

# 计算技术指标
if df is not None:
    df = calculate_technical_indicators(df)
    print(f"添加技术指标后的数据形状: {df.shape}")
    print("\\n特征列表:")
    print([col for col in df.columns if col not in ['open_time', 'close_time', 'symbol', 'interval']])


In [ ]:
# 数据预处理和数据集创建
def prepare_dataset(df, sequence_length=30, train_ratio=0.8):
    """统一的数据集准备函数，避免重复代码"""
    
    # 选择输入特征（移除原始OHLC作为输入，因为它们也是输出目标）
    feature_columns = [
        'rsi', 'ema_12', 'ema_26', 'macd', 'macd_signal', 'macd_histogram',
        'sma_10', 'sma_20', 'bb_upper', 'bb_lower', 'bb_position',
        'price_to_sma10', 'price_to_sma20', 'volatility', 'price_change',
        'volume_ratio', 'hl_ratio', 'oc_ratio'
    ]
    
    # 输出目标
    target_columns = ['open', 'high', 'low', 'close']
    
    # 处理缺失值
    df_clean = df.dropna()
    print(f"清理缺失值后的数据形状: {df_clean.shape}")
    
    # 准备特征和目标数据
    features = df_clean[feature_columns].values
    targets = df_clean[target_columns].values
    
    # 分别标准化特征和目标
    feature_scaler = MinMaxScaler()
    target_scaler = MinMaxScaler()
    
    features_scaled = feature_scaler.fit_transform(features)
    targets_scaled = target_scaler.fit_transform(targets)
    
    # 创建序列数据
    def create_sequences(features, targets, seq_length):
        X, y = [], []
        for i in range(seq_length, len(features)):
            X.append(features[i-seq_length:i])
            y.append(targets[i])
        return np.array(X), np.array(y)
    
    X, y = create_sequences(features_scaled, targets_scaled, sequence_length)
    
    # 划分训练和测试集
    split_index = int(len(X) * train_ratio)
    
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]
    
    print(f"训练集形状: X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"测试集形状: X_test: {X_test.shape}, y_test: {y_test.shape}")
    
    return (X_train, y_train, X_test, y_test, 
            feature_scaler, target_scaler, feature_columns, target_columns)

# 准备数据集
if df is not None:
    (X_train, y_train, X_test, y_test, 
     feature_scaler, target_scaler, feature_columns, target_columns) = prepare_dataset(df)


In [ ]:
# 优化的数据集类
class TimeSeriesDataset(Dataset):
    """优化的时间序列数据集类，支持动态加载"""
    def __init__(self, X, y, device=None):
        self.device = device if device else torch.device("cpu")
        # 避免一次性将所有数据移动到GPU，节省内存
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        # 在需要时才移动到设备
        return (self.X[idx].to(self.device), 
                self.y[idx].to(self.device))

# 创建数据集和数据加载器
train_dataset = TimeSeriesDataset(X_train, y_train, device)
test_dataset = TimeSeriesDataset(X_test, y_test, device)

# 使用更合理的批次大小
batch_size = min(64, len(train_dataset) // 10)  # 自适应批次大小
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"批次大小: {batch_size}")
print(f"训练批次数: {len(train_loader)}")
print(f"测试批次数: {len(test_loader)}")


In [ ]:
# 优化的LSTM模型
class OptimizedLSTM(nn.Module):
    """简化且更高效的LSTM模型"""
    def __init__(self, input_size, hidden_size=64, num_layers=2, output_size=4, dropout=0.2):
        super(OptimizedLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # 简化架构：使用2层双向LSTM + 注意力机制
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )
        
        # 注意力机制
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size * 2,
            num_heads=8,
            dropout=dropout,
            batch_first=True
        )
        
        # 输出层
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size * 2, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        
        # 残差连接的投影层
        self.projection = nn.Linear(input_size, output_size)
        
    def forward(self, x):
        # LSTM层
        lstm_out, _ = self.lstm(x)
        
        # 注意力机制
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)
        
        # 取最后一个时间步的输出
        last_hidden = attn_out[:, -1, :]
        
        # 全连接层
        out = self.dropout(last_hidden)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        # 残差连接（使用输入的最后一个时间步）
        residual = self.projection(x[:, -1, :])
        out = out + residual
        
        return out

# 创建模型
input_size = len(feature_columns)
model = OptimizedLSTM(
    input_size=input_size,
    hidden_size=64,
    num_layers=2,
    output_size=len(target_columns),
    dropout=0.2
).to(device)

# 计算模型参数数量
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"模型结构:")
print(model)
print(f"\\n总参数数: {total_params:,}")
print(f"可训练参数数: {trainable_params:,}")


In [ ]:
# 训练配置和优化器
class Config:
    """训练配置类"""
    def __init__(self):
        self.epochs = 100
        self.learning_rate = 0.001
        self.patience = 10  # 早停耐心值
        self.min_delta = 1e-6  # 最小改善阈值
        self.weight_decay = 1e-5  # L2正则化

config = Config()

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay)

# 学习率调度器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=5, verbose=True, min_lr=1e-6
)

# 早停类
class EarlyStopping:
    def __init__(self, patience=10, min_delta=1e-6):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = float('inf')
        
    def __call__(self, val_loss):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

early_stopping = EarlyStopping(patience=config.patience, min_delta=config.min_delta)

print("训练配置完成")


In [ ]:
# 优化的训练循环
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, early_stopping, config):
    """优化的训练函数，包含验证和早停"""
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    print("开始训练...")
    
    for epoch in range(config.epochs):
        # 训练阶段
        model.train()
        train_loss = 0.0
        train_batches = 0
        
        for batch_X, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            
            # 梯度裁剪防止梯度爆炸
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            optimizer.step()
            train_loss += loss.item()
            train_batches += 1
        
        avg_train_loss = train_loss / train_batches
        train_losses.append(avg_train_loss)
        
        # 验证阶段
        model.eval()
        val_loss = 0.0
        val_batches = 0
        
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
                val_batches += 1
        
        avg_val_loss = val_loss / val_batches
        val_losses.append(avg_val_loss)
        
        # 学习率调度
        scheduler.step(avg_val_loss)
        
        # 保存最佳模型
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
        
        # 每10个epoch打印一次进度
        if (epoch + 1) % 10 == 0:
            current_lr = optimizer.param_groups[0]['lr']
            print(f'Epoch [{epoch+1}/{config.epochs}] - '
                  f'训练损失: {avg_train_loss:.6f} - '
                  f'验证损失: {avg_val_loss:.6f} - '
                  f'学习率: {current_lr:.8f}')
        
        # 早停检查
        if early_stopping(avg_val_loss):
            print(f'\\n早停在epoch {epoch+1}，最佳验证损失: {best_val_loss:.6f}')
            break
    
    print(f'\\n训练完成！最佳验证损失: {best_val_loss:.6f}')
    
    # 加载最佳模型
    model.load_state_dict(torch.load('best_model.pth'))
    
    return train_losses, val_losses

# 开始训练
train_losses, val_losses = train_model(
    model, train_loader, test_loader, criterion, 
    optimizer, scheduler, early_stopping, config
)


In [ ]:
# 模型评估和可视化
def evaluate_model(model, test_loader, target_scaler, target_columns):
    """评估模型性能"""
    model.eval()
    predictions = []
    actuals = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            predictions.append(outputs.cpu().numpy())
            actuals.append(batch_y.cpu().numpy())
    
    # 合并所有批次的预测和实际值
    predictions = np.concatenate(predictions, axis=0)
    actuals = np.concatenate(actuals, axis=0)
    
    # 反标准化
    predictions_rescaled = target_scaler.inverse_transform(predictions)
    actuals_rescaled = target_scaler.inverse_transform(actuals)
    
    # 计算评估指标
    mse = mean_squared_error(actuals_rescaled, predictions_rescaled)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(actuals_rescaled, predictions_rescaled)
    
    # 计算MAPE (平均绝对百分比误差)
    mape = np.mean(np.abs((actuals_rescaled - predictions_rescaled) / actuals_rescaled)) * 100
    
    print("\n=== 模型评估结果 ===")
    print(f"均方误差 (MSE): {mse:.4f}")
    print(f"均方根误差 (RMSE): {rmse:.4f}")
    print(f"平均绝对误差 (MAE): {mae:.4f}")
    print(f"平均绝对百分比误差 (MAPE): {mape:.2f}%")
    
    # 按每个目标变量计算误差
    print("\n各目标变量的RMSE:")
    for i, col in enumerate(target_columns):
        col_rmse = np.sqrt(mean_squared_error(actuals_rescaled[:, i], predictions_rescaled[:, i]))
        print(f\"{col}: {col_rmse:.4f}\")
    
    return predictions_rescaled, actuals_rescaled

# 评估模型
predictions, actuals = evaluate_model(model, test_loader, target_scaler, target_columns)


In [ ]:
# 优化总结

## 原始代码的主要问题：

### 1. **重复的代码块**
- Cell 23和25中数据集创建代码完全重复
- 多次导入相同的包（pandas等）

### 2. **冗余的特征**
- 过多相关性高的移动平均线（3,6,12,20周期）
- 重复的布林带计算（bollinger_middle就是sma_20）
- 一些特征信息量低（如直接的差值特征）

### 3. **模型架构过度复杂**
- 使用4层LSTM容易过拟合
- MaxPool1d对时间序列意义不大
- 没有残差连接和注意力机制

### 4. **训练过程不完善**
- 缺少早停机制
- 没有学习率调度
- 缺少梯度裁剪
- 只打印最后一个batch的损失

## 优化后的改进：

### 1. **代码结构优化**
- 函数化所有主要操作
- 消除重复代码
- 添加错误处理

### 2. **特征工程优化**
- 使用相对比率代替绝对差值
- 减少冗余特征，保留信息量高的指标
- 添加成交量和波动率相关特征

### 3. **模型架构改进**
- 使用2层双向LSTM + 注意力机制
- 添加残差连接提升训练稳定性
- 合理的dropout设置

### 4. **训练优化**
- 早停机制防止过拟合
- 自适应学习率调度
- 梯度裁剪防止梯度爆炸
- 完整的验证循环和模型保存

### 5. **评估改进**
- 多种评估指标（MSE, RMSE, MAE, MAPE）
- 分别计算各目标变量的误差
- 反标准化后的真实值评估

这样的优化应该能显著提升模型性能和训练效率。
